In [ ]:
############################# - Notes for use to avoid confusion :)


#Not all cells work together. For example if you run the cell that deals with missing values by removing the entire row
#or removing all columns you will not be able to use the method to predict null values because the entries will be gone.

#So in order to predict the missing values after using the code cells that remove them you simply have to first run the code cell
#which reads the original data again

#To do:
    #Look into outliers
    #3D graphs
#############################

In [220]:
#missing values have value ? all for attribute 11

#either edible, poisonous or unknown. Unknowm and poisonous grouped together

#to deal with missing values use 'classifier' or 'regressor' based on values that arent missing to predict missing values in
#preprocessing. Be careful that vlaues using to predict arent overfitted. Compare to other methods such as removing entire attribute
#or giving deafult/mean value

from sklearn import metrics
from sklearn import model_selection
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

import umap.umap_ as umap

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [221]:
# read contents of data file
file = pd.read_csv("agaricus-lepiota.data", header = None)

# create list to be used as headers
features = ['edibility', 'capShape', 'capSurface', 'capColor', 'bruises', 'odor', 
            'gillAttachment', 'gillSpacing', 'gillSize', 'gillColor', 
            'stalkShape', 'stalkRoot', 'stalkSurfaceAboveRing', 'stalkSurfaceBelowRing', 
            'stalkColorAboveRing', 'stalkColorBelowRing', 'veilType', 'veilColor', 
            'ringNumber', 'ringType', 'sporePrintColor', 'population', 'habitat']

#convert '?' to NaN
file.replace({'?': np.nan}, inplace=True)

# converting data frame to csv
file.to_csv("agaricus-lepiota.csv", header=features, index=False)

data = pd.read_csv("agaricus-lepiota.csv")

#we can drop veilType as all entries are the same so cannot be used to predict
data.nunique()
data = data.drop('veilType', axis = 1).reset_index(drop=True)

#result = data.head(10)
#print(data)
#print(result)

In [ ]:
#removes column with missing values which in our data is only stalkRoot
#one way to deal with missing values
print(data.shape)

data = data.drop('stalkRoot', axis = 1).reset_index(drop=True)

print(data.shape)

data.to_csv("agaricus-lepiota-no-stalkRoot.csv")

print(data.head(10))

In [ ]:
#removes all rows that have missing values
print(data.shape)

data = data.dropna(axis = 0).reset_index(drop=True)
 
#shows we lose alot of data as 2480 rows lost
#bad way of handling missing values
print(data.shape)

data.to_csv("agaricus-lepiota-NaN-removed.csv")

In [222]:
#use label encoding on all featres that are categorical
#OneHot encoding not applicable as there are a high number of categories
le = LabelEncoder()

data['edibility'] = le.fit_transform(data['edibility'])
data['capShape'] = le.fit_transform(data['capShape'])
data['capSurface'] = le.fit_transform(data['capSurface'])
data['capColor'] = le.fit_transform(data['capColor'])
data['bruises'] = le.fit_transform(data['bruises'])
data['odor'] = le.fit_transform(data['odor'])
data['gillAttachment'] = le.fit_transform(data['gillAttachment'])
data['gillSpacing'] = le.fit_transform(data['gillSpacing'])
data['gillSize'] = le.fit_transform(data['gillSize'])
data['gillColor'] = le.fit_transform(data['gillColor'])
data['stalkShape'] = le.fit_transform(data['stalkShape'])
data['stalkRoot'] = le.fit_transform(data['stalkRoot'])
data['stalkSurfaceAboveRing'] = le.fit_transform(data['stalkSurfaceAboveRing'])
data['stalkSurfaceBelowRing'] = le.fit_transform(data['stalkSurfaceBelowRing'])
data['stalkColorAboveRing'] = le.fit_transform(data['stalkColorAboveRing'])
data['stalkColorBelowRing'] = le.fit_transform(data['stalkColorBelowRing'])
data['veilColor'] = le.fit_transform(data['veilColor'])
data['ringNumber'] = le.fit_transform(data['ringNumber'])
data['ringType'] = le.fit_transform(data['ringType'])
data['sporePrintColor'] = le.fit_transform(data['sporePrintColor'])
data['population'] = le.fit_transform(data['population'])
data['habitat'] = le.fit_transform(data['habitat'])

data.to_csv("agaricus-lepiota-encoded.csv")

In [223]:
#predict missing values for stalkRoot
NaN_predict = data.drop('edibility', axis = 1).reset_index(drop=True)

NaN_train_data = NaN_predict[NaN_predict['stalkRoot']!= 4].copy()
NaN_test_data = NaN_predict[NaN_predict['stalkRoot'] == 4].copy()

NaN_X_train = NaN_train_data.drop('stalkRoot', axis = 1).reset_index(drop=True)
NaN_y_train = NaN_train_data['stalkRoot']

NaN_X_test = NaN_test_data.drop('stalkRoot', axis = 1).reset_index(drop=True)

###############################################################################################################################
#This code further breaks the data down into training and test sets comprising of rows that have no missing values
#Allows us to run models and check how accurately they can predict the missing values then choose an appropriate one
#In this case both Random Forest and Support Vector Classification predict that exact same values for 'stalkRoot' so either can be used
#Both have 100% accuracy in predicitng 'stalkRoot' so we know when we predict the missing values they will most likely be correct.
X_train, X_test, y_train, y_test = train_test_split(NaN_train_data, NaN_train_data['stalkRoot'], test_size=0.2,random_state=42)

rfr = RandomForestRegressor()
svm = SVC()

rfr.fit(X_train, y_train)
rfr_pred = rfr.predict(X_test)

svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)

rfr_rounded = (np.rint(rfr_pred)).astype(int)

svm_rounded = (np.rint(svm_pred)).astype(int)

different = []

for i in range(len(rfr_rounded)):
    if rfr_rounded[i] != svm_rounded[i]:
        different.append(rfr_rounded[i], svm_rounded[i])

print(len(different))

print(classification_report(y_test,rfr_rounded))

print(classification_report(y_test,svm_rounded))
##############################################################################################################################

rfc = RandomForestRegressor()

rfc.fit(NaN_X_train,NaN_y_train)
y_pred = rfc.predict(NaN_X_test)

rounded_data = (np.rint(y_pred)).astype(int)
NaN_test_data['stalkRoot'] = rounded_data

NaN_test_data.to_csv("predicted-missing-values.csv")

frames = [NaN_train_data, NaN_test_data]
treated_data = pd.concat(frames)
treated_data.insert(0, column = 'edibility', value = data['edibility'])

X = treated_data.drop('edibility', axis = 1).reset_index(drop=True)
y = treated_data['edibility']

data_scaled = StandardScaler().fit_transform(X)

treated_data.to_csv("full-data.csv")

0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       724
           1       1.00      1.00      1.00       119
           2       1.00      1.00      1.00       239
           3       1.00      1.00      1.00        47

    accuracy                           1.00      1129
   macro avg       1.00      1.00      1.00      1129
weighted avg       1.00      1.00      1.00      1129

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       724
           1       1.00      1.00      1.00       119
           2       1.00      1.00      1.00       239
           3       1.00      1.00      1.00        47

    accuracy                           1.00      1129
   macro avg       1.00      1.00      1.00      1129
weighted avg       1.00      1.00      1.00      1129



In [ ]:
#variance inflation factor

fig = plt.figure(figsize=(25, 20))
sns.heatmap(X.corr(), annot=True)
plt.savefig("Heatmap.png")

#as you can see from heatmap there are some fearutes with high correlation

vif_data = pd.DataFrame()
vif_data['feature'] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
						for i in range(len(X.columns))]

print(vif_data)

features_to_drop = []

for i in range(len(X.columns)):
	if vif_data["VIF"][i] > 7:
		drop = vif_data["VIF"][i]
		features_to_drop.append(vif_data["feature"][i])
	
new_X = X.drop(features_to_drop, axis = 1).reset_index(drop=True)

#generally features with VIF > 5 get removed but we can experiment with slightly higher i.e > 7

fig2 = plt.figure(figsize=(25, 20))
sns.heatmap(new_X.corr(), annot=True)
plt.savefig("Heatmap2.png")

#as seen by second heatmap these have much less colinearality

In [ ]:
#data visualisation pca
#a linear method so less complex than tsne thus takes much less time
#not very useful to visualise our dataset I dont think as shown by the explained variance and scree plot
pca = PCA()
pca_features = pca.fit_transform(data_scaled)

per_var = np.round(pca.explained_variance_ratio_*100, decimals = 1)
print(per_var)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]

#to be seen on a 2D-plot we can only use PC1 and PC2 which gives us the highest explained variance but still very low
#around 19% + 12% = 31% explained variance which is bad
plt.bar(x= range(1,len(per_var)+1), height = per_var, tick_label = labels)
plt.show()

pca_df = pd.DataFrame({'pca_1': pca_features[:,0], 'pca_2': pca_features[:,1], 'label': treated_data['edibility']})
print(pca_df)

sns.scatterplot(x = 'pca_1', y = 'pca_2', hue = 'label', data = pca_df)

plt.show()

#PCA could be useful to reduce multicolinearality though instead of data visualisation
#use the first 8 pc's to get about ~90% explained variance, 9 to get ~93%, 10 to get ~95%

In [ ]:
#data visualisation t-sne
#unlike pca its not linear so much more complex and better for our data I think
#result of t-sne greatly affected by 'perplexity' hyperparameter
#try perplexity values in the range 5-100 default is 30. Higher perplexity takes longer but may be better
n_components = 2
tsne = TSNE(n_components, perplexity=5)
tsne_features = tsne.fit_transform(data_scaled)

tsne_df = pd.DataFrame({'tsne_1': tsne_features[:,0], 'tsne_2': tsne_features[:,1], 'label': treated_data['edibility']})

sns.scatterplot(x = 'tsne_1', y = 'tsne_2', hue = 'label', data = tsne_df)

plt.show()

In [ ]:
#data visualisation UMAP
#UMAP works very similarly to tsne but get more efficient for higher volumes of data
#result greatly affected by n_neighbors and min_dist hyperparameters
#for n_neighbors try range 5-100 and min_dist 0.1-1
fit = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=2,
    metric = 'correlation'
)

umap_features = fit.fit_transform(data_scaled)

umap_df = pd.DataFrame({'umap_1': umap_features[:,0], 'umap_2': umap_features[:,1], 'label': treated_data['edibility']})

sns.scatterplot(x = 'umap_1', y = 'umap_2', hue = 'label', data = umap_df)

plt.show()